In [21]:
from datasets import load_dataset
import pandas as pd
import numpy as np
import en_core_web_sm

In [3]:
nlp = en_core_web_sm.load()
# stats_cal(datasets_test)


In [22]:
# # load cnn_dailymail
# datasets_train = load_dataset("cnn_dailymail", "3.0.0", split="train")
# datasets_test = load_dataset("cnn_dailymail", "3.0.0", split="test")
# datasets_valid = load_dataset("cnn_dailymail", "3.0.0", split="validation")
# # load xsum
# xsum_train = load_dataset("xsum", "1.2.0", split="train")
# xsum_test = load_dataset("xsum", "1.2.0", split="test")
# xsum_valid = load_dataset("xsum", "1.2.0", split="validation")

# load wiki_lingua English
wiki_lingua_train = load_dataset("wiki_lingua", "english", split="train")

# # load scitldr
# scitldr_train = load_dataset("scitldr", "Abstract", split="train")
# scitldr_test = load_dataset("scitldr", "Abstract", split="test")
# scitldr_valid = load_dataset("scitldr", "Abstract", split="validation")

# # load billsum
# billsum_train = load_dataset("billsum", "3.0.0", split="train")
# billsum_test = load_dataset("billsum", "3.0.0", split="test")
# billsum_valid = load_dataset("billsum", "3.0.0", split="ca_test")

Generating train split:   0%|          | 0/57945 [00:00<?, ? examples/s]

Dataset wiki_lingua downloaded and prepared to /home/jli/working_dir/datasets_cache/wiki_lingua/english/1.1.1/dad5f0710d1bb1a2ed1eb7633802fe147ebd27a3812985f5641bb7aca6ba5c08. Subsequent calls will reuse this data.


In [23]:
df = pd.DataFrame(wiki_lingua_train["article"])

In [24]:
df["url"] = pd.DataFrame(wiki_lingua_train["url"])

In [25]:
df.document = df.document.str.join("")
df.summary = df.summary.str.join("")

In [26]:
df.replace(to_replace=r'^\s*$',value=np.nan,regex=True,inplace=True)

In [28]:
df.isnull().sum()

section_name        0
document        10685
summary         10678
url                 0
dtype: int64

In [18]:
df[df.isnull().values == True]

,section_name,document,summary,url
14,[],NaN,NaN,https://www.wikihow.com/Flick-a-Card
14,[],NaN,NaN,https://www.wikihow.com/Flick-a-Card
15,[],NaN,NaN,https://www.wikihow.com/Replace-a-Flat-Roof
15,[],NaN,NaN,https://www.wikihow.com/Replace-a-Flat-Roof
21,[],NaN,NaN,https://www.wikihow.com/Create-and-Work-a-Juxl...
...,...,...,...,...
57900,[],NaN,NaN,https://www.wikihow.com/Make-the-Most-of-Your-...
57918,[],NaN,NaN,https://www.wikihow.com/Use-Extended-Desktop-V...
57918,[],NaN,NaN,https://www.wikihow.com/Use-Extended-Desktop-V...
57924,[],NaN,NaN,https://www.wikihow.com/Kill-Household-Pests-(...


In [20]:
wiki_lingua_train["url"][2]


'https://www.wikihow.com/Calculate-Annualized-Portfolio-Return'

In [7]:
wiki_lingua_train["article"][14]

{'section_name': [], 'document': [], 'summary': []}

In [ ]:
def rename_datasets(dataset):
    dataset = dataset.rename_column(dataset.column_names[0],"source")
    dataset = dataset.rename_column(dataset.column_names[1],"target")
    return dataset